In [1]:
import pybaseball as pyb
import pandas as pd

In [2]:
batting_df = pd.read_csv('fangraphs_batting_stats.csv')
pitching_df = pd.read_csv('fangraphs_pitching_stats.csv')
team_standings_df = pd.read_csv('mlb_team_standings.csv')
mvp_df = pd.read_csv('mvp_winners_2000_2024.csv')

# Look at column names, data types, missing values
print("\n--- Batting DataFrame Info ---")
batting_df.info()
print("\n--- Pitching DataFrame Info ---")
pitching_df.info()
#print("\n--- Team Standings DataFrame Info ---")
#team_standings_df.info()
print("\n--- MVP Winners DataFrame Info ---")
mvp_df.info()


--- Batting DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32465 entries, 0 to 32464
Columns: 320 entries, IDfg to L-WAR
dtypes: float64(291), int64(25), object(4)
memory usage: 79.3+ MB

--- Pitching DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17698 entries, 0 to 17697
Columns: 393 entries, IDfg to Pit+ FO
dtypes: float64(362), int64(27), object(4)
memory usage: 53.1+ MB

--- MVP Winners DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Season  50 non-null     int64 
 1   Name    50 non-null     object
 2   League  50 non-null     object
 3   Team    50 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.7+ KB


In [3]:
print(f"\nDuplicates in batting_df: {batting_df.duplicated().sum()}")
print(f"Duplicates in pitching_df: {pitching_df.duplicated().sum()}")
print(f"Duplicates in team_standings_df: {mvp_df.duplicated().sum()}")



Duplicates in batting_df: 0
Duplicates in pitching_df: 0
Duplicates in team_standings_df: 0


Now we need to merge these DataFrames.

In [4]:
# Step 1: MVP flag
mvp_df['MVP'] = True
mvp_flag = mvp_df[['Name', 'Season', 'MVP']]

# Step 2: Merge MVP into batting & pitching separately
batting_df = batting_df.merge(mvp_flag, on=['Name', 'Season'], how='left')
pitching_df = pitching_df.merge(mvp_flag, on=['Name', 'Season'], how='left')

# Step 3: Fill NaN with False
batting_df['MVP'] = batting_df['MVP'].fillna(False)
pitching_df['MVP'] = pitching_df['MVP'].fillna(False)

# Step 4: Merge batting + pitching using an outer join
# Let pandas handle duplicate columns by adding suffixes
bat_pitch_df = pd.merge(
    batting_df,
    pitching_df,
    on=['IDfg', 'Season'],
    how='outer',
    suffixes=('_bat', '_pitch') 
)

# Step 5: Consolidate the MVP, Name, and Team columns
# Coalesce the MVP columns: if either MVP_bat or MVP_pitch is True, the player is an MVP.
bat_pitch_df['MVP'] = bat_pitch_df['MVP_bat'].fillna(False) | bat_pitch_df['MVP_pitch'].fillna(False)

# Coalesce the Name and Team columns, taking the value from the batting stats first.
bat_pitch_df['Name'] = bat_pitch_df['Name_bat'].fillna(bat_pitch_df['Name_pitch'])
bat_pitch_df['Team'] = bat_pitch_df['Team_bat'].fillna(bat_pitch_df['Team_pitch'])

# Step 6: Drop the old, suffixed columns
bat_pitch_df = bat_pitch_df.drop(columns=[
    'Name_bat', 'Name_pitch', 
    'Team_bat', 'Team_pitch', 
    'MVP_bat', 'MVP_pitch'
])

/var/folders/pz/pxdv_rx91j70b52l206jy3qr0000gn/T/ipykernel_3295/4052506622.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  batting_df['MVP'] = batting_df['MVP'].fillna(False)
/var/folders/pz/pxdv_rx91j70b52l206jy3qr0000gn/T/ipykernel_3295/4052506622.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pitching_df['MVP'] = pitching_df['MVP'].fillna(False)
/var/folders/pz/pxdv_rx91j70b52l206jy3qr0000gn/T/ipykernel_3295/4052506622.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future 

Now we should have all of the players merged together with a new column for if they won the MVP award or not. 

In [28]:
(bat_pitch_df.head())

,IDfg,Season,Age_bat,G_bat,AB,PA,H_bat,1B,2B,3B,...,Pit+ FS,Stuff+,Location+,Pitching+,Stf+ FO,Loc+ FO,Pit+ FO,MVP,Name,Team
0,1,2002,24.0,12.0,13.0,13.0,7.0,5.0,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Alfredo Amezaga,ANA
1,1,2003,25.0,37.0,105.0,120.0,22.0,15.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Alfredo Amezaga,ANA
2,1,2004,26.0,73.0,93.0,105.0,15.0,11.0,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Alfredo Amezaga,ANA
3,1,2005,27.0,6.0,8.0,9.0,2.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Alfredo Amezaga,- - -
4,1,2006,28.0,132.0,334.0,378.0,87.0,72.0,9.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Alfredo Amezaga,FLA
